In [18]:
import os
from thirdai import bolt, dataset
from thirdai.dataset import InMemorySparseDataset as sp
from thirdai.dataset import DataPipeline, blocks
import numpy as np

In [6]:
data=[]
with open("format_svm",'r') as f:
    for lines in f.readlines():
        lines=lines.rstrip('\n').split(" ")
        lines=[lines[0]+':0.1,522:3.3']+lines[1:]
        data.append('\t'.join(lines))



In [7]:
print(data[0])

302:0.1,522:3.3	56460:1	61289:1	73380:1	25792:1	99240:1	87955:1	88437:1	25792:1	8301:1	19168:1	61276:1	14367:1	5400:1	94746:1	55217:1	43687:1	50507:1	10231:1	13676:1	33400:1	17109:1	27032:1	80443:1	40260:1	11063:1	24121:1	14537:1	15268:1	63441:1	80994:1	95157:1	85968:1	91564:1	80994:1	69460:1	22340:1	9983:1	17802:1	58360:1	43299:1	99079:1	98780:1	63306:1	62173:1	62052:1	50488:1	88881:1	26052:1	58831:1	14999:1	42252:1	89393:1


In [8]:
with open("weight_multi_svm","w") as f:
    for lines in data:
        f.write(lines+'\n')

In [19]:
from thirdai.dataset import text_encodings, categorical_encodings,weighted_categorical_encodings
n_classes=1000
pipeline = DataPipeline(
    filename="multi_svm", 
    input_blocks=[
        blocks.Text(col=1, encoding=text_encodings.UniGram(dim=100_000)), # Default encoding for text block
        blocks.Text(col=1, encoding=text_encodings.PairGram(dim=100_000)),
        blocks.Text(col=1, encoding=text_encodings.CharKGram(k=3, dim=100_000)), # Character trigrams
        blocks.Categorical(col=0, encoding=categorical_encodings.ContiguousNumericId(dim=2,delimiter=',')), # Default encoding for categorical block
    ],
    label_blocks=[blocks.Categorical(col=0, dim=n_classes,delimiter=',')],
    batch_size=256,
    has_header=False, # Pipeline discards the header if the dataset has one.
    delimiter='\t') # Any character is a valid delimiter.

In [20]:
from thirdai.dataset import text_encodings, categorical_encodings,weighted_categorical_encodings
n_classes=1000
wpipeline = DataPipeline(
    filename="weight_multi_svm", 
    input_blocks=[
        blocks.Text(col=1, encoding=text_encodings.UniGram(dim=100_000)), # Default encoding for text block
        blocks.Text(col=1, encoding=text_encodings.PairGram(dim=100_000)),
        blocks.Text(col=1, encoding=text_encodings.CharKGram(k=3, dim=100_000)), # Character trigrams
        blocks.WeightedCategorical(col=0, encoding=weighted_categorical_encodings.WeightedContiguousNumericId(dim=2,delimiter=',')), # Default encoding for categorical block
    ],
    label_blocks=[blocks.WeightedCategorical(col=0, dim=n_classes,delimiter=',')],
    batch_size=256,
    has_header=False, # Pipeline discards the header if the dataset has one.
    delimiter='\t') # Any character is a valid delimiter.

In [26]:
data,label=wpipeline.load_in_memory()

In [27]:
print(label[0][0])

[(302, 0.1), (522, 3.3)]


In [28]:
print(data)

In [31]:
print(wpipeline.get_label_dim())

1000


In [32]:
layers = [
        bolt.FullyConnected(
            dim=100,
            sparsity=1,
            activation_function="ReLU",
        ),
        bolt.FullyConnected(dim=wpipeline.get_label_dim(), activation_function="Softmax"),
    ]
network = bolt.Network(layers=layers, input_dim=wpipeline.get_input_dim())

Initializing Bolt network...
========= Bolt Network =========
InputLayer (Layer 0): dim=300002
FullyConnectedLayer (Layer 1): dim=100, sparsity=1, act_func=ReLU
FullyConnectedLayer (Layer 2): dim=1000, sparsity=1, act_func=Softmax
Initialized Network in 0 seconds


In [33]:
network.train(data,
        label,
        bolt.CategoricalCrossEntropyLoss(),
        0.1,
        5,
        rehash=3000,
        rebuild=10000,
        metrics=[],
        verbose=False,)

{'epoch_times': [0.0, 0.0, 0.0, 0.0, 0.0]}